In [1]:
import pickle
import os
from od.social import SocialGroup
from od.network.types import ResourceAllocatorType,BaseStationType
from od.misc.interest import InterestConfig
from od.config import BS_PRESET,SUMO_SIM_SECONDS

In [4]:
statistics  = [None for _ in ResourceAllocatorType]
for _res_alloc, res_alloc in enumerate(ResourceAllocatorType):
    interest_config = InterestConfig(res_alloc,True,5,4419883)
    print(interest_config)
    with open("data/{}/{}/object.pickle".format(interest_config.rng_seed,str(interest_config)), "rb") as file:
                       statistics[_res_alloc] = pickle.load(file)

res_alloc_type(OMA) req_rsu(True) appdata_poisson(5)
res_alloc_type(NOMA_OPT) req_rsu(True) appdata_poisson(5)


In [5]:
class GV:
    NET_STATION_CONTROLLER = [
            bs_config for bs_config in BS_PRESET.values()
             if ((bs_config["type"] == BaseStationType.UMA)  or 
            (interest_config.req_rsu and bs_config["type"] == BaseStationType.UMI))
    ]
    class STATISTIC:
        def Log(data):
            print(data)

In [6]:
GV.NET_STATION_CONTROLLER

[{'color': (0, 0, 0, 255),
  'pos': (285, 245),
  'img': '/home/OD/repos/ProjectSumo/wifi.png',
  'width': 5,
  'height': 3.9,
  'type': <BaseStationType.UMA: 1>},
 {'color': (0, 0, 0, 255),
  'pos': (127, 217),
  'img': '/home/OD/repos/ProjectSumo/wifi.png',
  'width': 5,
  'height': 3.9,
  'type': <BaseStationType.UMI: 0>},
 {'color': (0, 0, 0, 255),
  'pos': (52, 145),
  'img': '/home/OD/repos/ProjectSumo/wifi.png',
  'width': 5,
  'height': 3.9,
  'type': <BaseStationType.UMI: 0>},
 {'color': (0, 0, 0, 255),
  'pos': (315, 395),
  'img': '/home/OD/repos/ProjectSumo/wifi.png',
  'width': 5,
  'height': 3.9,
  'type': <BaseStationType.UMI: 0>},
 {'color': (0, 0, 0, 255),
  'pos': (270, 50),
  'img': '/home/OD/repos/ProjectSumo/wifi.png',
  'width': 5,
  'height': 3.9,
  'type': <BaseStationType.UMI: 0>},
 {'color': (0, 0, 0, 255),
  'pos': (525, 95),
  'img': '/home/OD/repos/ProjectSumo/wifi.png',
  'width': 5,
  'height': 3.9,
  'type': <BaseStationType.UMI: 0>}]

In [7]:
for sg in SocialGroup:
    for header in statistics[0].sg_header[sg].keys():
        if(header not in statistics[1].sg_header[sg]):
            print("{} in statistic[0] but not in statistic[1]".format(header))
        else:
            s0 = statistics[0].sg_header[sg][header].time_bs_serv
            s1 = statistics[1].sg_header[sg][header].time_bs_serv
            for bs_serial,bs_config in enumerate(GV.NET_STATION_CONTROLLER):
                if(s0[bs_serial] != s1[bs_serial]):
                    print(
                        "{}/{}/{}: {}={},{}={}".format(
                            sg,
                            header,
                            bs_serial,
                            ResourceAllocatorType(0).name,
                            s0[bs_serial],
                            ResourceAllocatorType(1).name,
                            s1[bs_serial]
                        )
                    )

CRITICAL/0.108-75/5: OMA=259.109,NOMA_OPT=259.11
CRITICAL/0.149-84/2: OMA=259.27,NOMA_OPT=259.27099999999996
CRITICAL/0.108-149/5: OMA=267.117,NOMA_OPT=267.118
CRITICAL/0.185-11/0: OMA=278.605,NOMA_OPT=278.606
CRITICAL/1.126-322/1: OMA=284.134,NOMA_OPT=284.135
CRITICAL/0.149-402/4: OMA=288.14,NOMA_OPT=288.141
CRITICAL/1.171-321/0: OMA=288.634,NOMA_OPT=288.635
CRITICAL/0.140-395/3: OMA=291.141,NOMA_OPT=291.142
CRITICAL/0.167-420/3: OMA=293.143,NOMA_OPT=293.144
CRITICAL/0.185-375/2: OMA=314.64,NOMA_OPT=314.641
CRITICAL/1.180-441/3: OMA=316.148,NOMA_OPT=316.149
CRITICAL/1.171-622/1: OMA=319.665,NOMA_OPT=319.666
CRITICAL/0.194-335/2: OMA=321.134,NOMA_OPT=321.135
CRITICAL/0.140-703/3: OMA=323.174,NOMA_OPT=323.175
GENERAL/1.126-12/0: OMA=253.10500000000002,NOMA_OPT=253.10300000000004
GENERAL/0.108-12/0: OMA=253.10600000000002,NOMA_OPT=253.10500000000002
GENERAL/0.140-10/0: OMA=253.10600000000002,NOMA_OPT=253.10500000000002
GENERAL/0.117-13/0: OMA=253.10700000000003,NOMA_OPT=253.1060000000000

In [8]:
 def BaseStationThroughPutReport(self):
        sg_stats = {}
        bs_bits = [0 for _ in GV.NET_STATION_CONTROLLER]
        GV.STATISTIC.Log("=====BaseStationThroughPutReport=====")
        for sg in SocialGroup:
            for record in self.sg_header[sg].values():
                for bs_serial,bs_config in enumerate(GV.NET_STATION_CONTROLLER):
                    if(record.time_bs_serv[bs_serial] > 0):
                        bs_bits[bs_serial] += record.bits
        for bs_type in BaseStationType:
            bs_type_num = 0
            bs_type_bits = 0
            for bs_serial in [bs_serial for bs_serial,bs_config in enumerate(GV.NET_STATION_CONTROLLER) if bs_config["type"] == bs_type]:
                bs_type_bits += bs_bits[bs_serial]
                bs_type_num += 1
            bs_type_through_put_avg = (
                (bs_type_bits / max(bs_type_num, 1)) /
                SUMO_SIM_SECONDS
            )
            print("====={}=====".format(bs_type.name))
            print("Through Put:{:.2f}/s".format(bs_type_through_put_avg))
            sg_stats[bs_type] = bs_type_through_put_avg
        return sg_stats

In [98]:
BaseStationThroughPutReport(statistic)

=====BaseStationThroughPutReport=====
=====UMI=====
Through Put:456557.22/s
=====UMA=====
Through Put:697338.22/s


{<BaseStationType.UMI: 0>: 456557.22222222225,
 <BaseStationType.UMA: 1>: 697338.2222222222}